# Environments


**Note:** this notebook uses the bash kernel. Intall it with `pip install bash_kernel` or run these commands in a shell.


A "virtualenv" is a directory isolating a collection of Python packages.
The `venv` package in the standard library creates virtualenvs.
There are various wrappers, such as `virtualenvwrapper` that can add some functionality if you use envs a lot.


to create an env:

In [1]:
# --clear means delete an env if there already was one at this location
python -m venv --clear ./env-1
./env-1/bin/pip install --upgrade pip

# tree will show us a peek at what's in the env
tree -L 2 env-1

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
env-1
├── bin
│   ├── Activate.ps1
│   ├── activate
│   ├── activate.csh
│   ├── activate.fish
│   ├── pip
│   ├── pip3
│   ├── pip3.10
│   ├── python -> /Users/minrk/conda/bin/python
│   ├── python3 -> python
│   └── python3.10 -> python
├── include
├── lib
│   └── python3.10
└── pyvenv.cfg

4 directories, 11 files


I have a lot of packages in my main environment!

In [2]:
pip list | wc -l

     486


Run


```bash
source $PREFIX/bin/active
```

to *activate* an env. Which mainly means:

1. put the env on the front of your $PATH
2. now, when I run `python` or `pip`, only the packages in the env are available:

In [3]:
VIRTUAL_ENV_DISABLE_PROMPT=1
source ./env-1/bin/activate
pip list

Package    Version
---------- -------
pip        22.3.1
setuptools 58.1.0


In [4]:
cat requirements.txt

altair
flask
vega_datasets


In [5]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 20.4 MB/s eta 0:00:00m eta 0:00:01
  Using cached Flask-2.2.2-py3-none-any.whl (101 kB)
  Using cached vega_datasets-0.9.0-py3-none-any.whl (210 kB)
  Using cached numpy-1.23.4-cp310-cp310-macosx_11_0_arm64.whl (13.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 52.3 MB/s eta 0:00:000:00:01 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.0 MB/s eta 0:00:00
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached jsonschema-4.17.0-py3-none-any.whl (83 kB)
  Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)
  Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached MarkupSafe-2.1.1-cp310-cp310-macosx_10_9_universal2.whl (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 10.0 MB/s eta 0:00:00
  Using cached attrs-22.1.0-py2.py3-none-any

In [6]:
pip list

Package         Version
--------------- -------
altair          4.2.0
attrs           22.1.0
click           8.1.3
entrypoints     0.4
Flask           2.2.2
itsdangerous    2.1.2
Jinja2          3.1.2
jsonschema      4.17.0
MarkupSafe      2.1.1
numpy           1.23.4
pandas          1.5.1
pip             22.3.1
pyrsistent      0.19.2
python-dateutil 2.8.2
pytz            2022.6
setuptools      58.1.0
six             1.16.0
toolz           0.12.0
vega-datasets   0.9.0
Werkzeug        2.2.2


But what about reproducible builds? What if a new release of altair breaks my application?

Solution: Pin specific versions?

```
# requirements.txt
altair==4.1.0
vega-datasets==0.8.0
flask==1.1.2
```

💣 this is the works possible thing!

1. it ensures that your direct dependencies are not updated (fine), but
2. it **does not** ensure that their dependencies are not updated

This *guarantees* that your env will break when a dependency is updated. If you had left everything unpinned, it is likely your code would not break, unless the APIs your code uses change. But partial pinning is a way to *guarantee* something will break, even if your code works fine with the latest version of everything.

If you are pinning dependencies, it should be *all or nothing*, never partial. Ideally, this should include Python itself!

In [ ]:
pip freeze

[pip-tools](https://github.com/jazzband/pip-tools) is a collection of tools to solve
the "loose vs pinned" dependency problem.

Instead of a single `requirements.txt`, you have a human-mananged `requirements.in` with only direct, loose dependencies, and an automatically managed `requirements.txt` with a fully pinned environment, including all dependencies.

In [7]:
pip install pip-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.5 MB/s eta 0:00:00
  Using cached pep517-0.13.0-py3-none-any.whl (18 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)


In [17]:
cp requirements.txt pip-tools/requirements.in
rm -f pip-tools/requirements.txt

In [18]:
cd pip-tools

In [19]:
pip-compile

#
# This file is autogenerated by pip-compile with python 3.10
# To update, run:
#
#    pip-compile
#
altair==4.2.0
    # via -r requirements.in
attrs==22.1.0
    # via jsonschema
click==8.1.3
    # via flask
entrypoints==0.4
    # via altair
flask==2.2.2
    # via -r requirements.in
itsdangerous==2.1.2
    # via flask
jinja2==3.1.2
    # via
    #   altair
    #   flask
jsonschema==4.17.0
    # via altair
markupsafe==2.1.1
    # via
    #   jinja2
    #   werkzeug
numpy==1.23.4
    # via
    #   altair
    #   pandas
pandas==1.5.1
    # via
    #   altair
    #   vega-datasets
pyrsistent==0.19.2
    # via jsonschema
python-dateutil==2.8.2
    # via pandas
pytz==2022.6
    # via pandas
six==1.16.0
    # via python-dateutil
toolz==0.12.0
    # via altair
vega-datasets==0.9.0
    # via -r requirements.in
werkzeug==2.2.2
    # via flask


When you start, there is no difference between

```bash
pip install -r requirements.in
```

and

```bash
pip install -r requirements.txt
```

The distinction is: `requirements.in` will install *different things over time*, while `requirements.txt` will always install the same exact things until you change it, e.g. by running `pip-compile` again to upgrade packages, or after editing `requirements.in`.


In [20]:
cat requirements.in

altair
flask
vega_datasets


In [21]:
cat requirements.txt

#
# This file is autogenerated by pip-compile with python 3.10
# To update, run:
#
#    pip-compile
#
altair==4.2.0
    # via -r requirements.in
attrs==22.1.0
    # via jsonschema
click==8.1.3
    # via flask
entrypoints==0.4
    # via altair
flask==2.2.2
    # via -r requirements.in
itsdangerous==2.1.2
    # via flask
jinja2==3.1.2
    # via
    #   altair
    #   flask
jsonschema==4.17.0
    # via altair
markupsafe==2.1.1
    # via
    #   jinja2
    #   werkzeug
numpy==1.23.4
    # via
    #   altair
    #   pandas
pandas==1.5.1
    # via
    #   altair
    #   vega-datasets
pyrsistent==0.19.2
    # via jsonschema
python-dateutil==2.8.2
    # via pandas
pytz==2022.6
    # via pandas
six==1.16.0
    # via python-dateutil
toolz==0.12.0
    # via altair
vega-datasets==0.9.0
    # via -r requirements.in
werkzeug==2.2.2
    # via flask


## pipenv

[pipenv](https://pipenv.pypa.io) is a tool built around pipfiles that does a similar thing to pip-tools, but one step removed.

- uses [Pipfile](https://github.com/pypa/pipfile) format instead of requirements.txt
- Pipfile.lock for pinned versions
- includes specifying the Python version itself, and additional informat about *how* to install pacakges
- manages environments as well


In [22]:
cd ../pipfile

In [24]:
cat Pipfile

[requires]
python_version = "3.10"

[packages]
altair = "*"
flask = "*"
vega-datasets = "*"


Leave our earlier env

In [25]:
deactivate
pip install pipenv

In [26]:
pipenv lock

Creating a virtualenv for this project...
Pipfile: /Users/minrk/dev/simula/in3110/site/lectures/12-production/pipfile/Pipfile
Using /opt/homebrew/bin/python3 (3.10.8) to create virtualenv...
⠇ Creating virtual environment...created virtual environment CPython3.10.8.final.0-64 in 391ms
  creator CPython3Posix(dest=/Users/minrk/env/pipfile-K1fiq9Jh, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/Users/minrk/Library/Application Support/virtualenv)
    added seed packages: pip==22.3.1, setuptools==65.5.1, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator

✔ Successfully created virtual environment! 
Virtualenv location: /Users/minrk/env/pipfile-K1fiq9Jh
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [dev-packages] dependencies...
Updated Pipfile.lock (70

In [27]:
pipenv install

Installing dependencies from Pipfile.lock (70b772)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [28]:
pipenv run python -c "import sys; print(sys.prefix)"

/Users/minrk/env/pipfile-K1fiq9Jh


In [29]:
pipenv run pip list

Package         Version
--------------- -------
altair          4.2.0
attrs           22.1.0
click           8.1.3
entrypoints     0.4
Flask           2.2.2
itsdangerous    2.1.2
Jinja2          3.1.2
jsonschema      4.17.0
MarkupSafe      2.1.1
numpy           1.23.4
pandas          1.5.1
pip             22.3.1
pyrsistent      0.19.2
python-dateutil 2.8.2
pytz            2022.6
setuptools      65.5.1
six             1.16.0
toolz           0.12.0
vega-datasets   0.9.0
Werkzeug        2.2.2
wheel           0.37.1


In [30]:
head -n 30 Pipfile.lock

{
    "_meta": {
        "hash": {
            "sha256": "644e57b07835e87b285d8338676a8bde3d24b8412120203e96731a83b770b772"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_version": "3.10"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "altair": {
            "hashes": [
                "sha256:0c724848ae53410c13fa28be2b3b9a9dcb7b5caa1a70f7f217bd663bb419935a",
                "sha256:d87d9372e63b48cd96b2a6415f0cf9457f50162ab79dc7a31cd7e024dd840026"
            ],
            "index": "pypi",
            "version": "==4.2.0"
        },
        "attrs": {
            "hashes": [
                "sha256:29adc2665447e5191d0e7c568fde78b21f9672d344281d0c6e1ab085429b22b6",
                "sha256:86efa402f67bf2df34f51a335487cf46b1ec130d02b8d39fd248abfd30da551c"
